## Домашняя работа №5. Рекуррентные сети

### Данные

In [59]:
def caesar(plaintext, shift):
    alphabet = 'абвгдеёжзийклмнопрстуфхцчшщъыьэюя'
    shifted_alphabet = alphabet[shift:] + alphabet[:shift]
    table = str.maketrans(alphabet, shifted_alphabet)
    return plaintext.translate(table)

In [60]:
import re
from random import randint

data = []
encoded = []
encoded_rand = []
SIGNS = set('IVXL')

with open('onegin.txt', mode='r', encoding="utf8") as f:
    for line in f:
        if (len(line.strip()) < 8) and (line[0] in SIGNS):
            f.readline()
            for _ in range(14):
                sent = ' '.join(re.findall(r'[а-яё]+', f.readline().lower()))
                if len(sent) == 0:
                    break
                data.append(sent)
                encoded.append(caesar(sent, 2))
                encoded_rand.append(caesar(sent, randint(2, 5)))
        if len(data) >= 14 * 10:
            break

In [61]:
data[0], caesar(data[0], 1)

('мой дядя самых честных правил', 'нпк еаеа тбньц шётуоьц рсбгйм')

In [62]:
CHARS = set('абвгдеёжзийклмнопрстуфхцчшщъыьэюя ')
INDEX_TO_CHAR = ['none'] + [w for w in CHARS]
CHAR_TO_INDEX = {w: i for i, w in enumerate(INDEX_TO_CHAR)}

In [63]:
len(INDEX_TO_CHAR)

35

In [64]:
len(max(data, key=lambda x: len(x)))

40

In [65]:
import torch

In [66]:
MAX_LEN = 40
X = torch.zeros((len(data), MAX_LEN), dtype=int)
XR = torch.zeros((len(data), MAX_LEN), dtype=int)
Y = torch.zeros((len(data), MAX_LEN), dtype=int)
for i in range(len(data)):
    for j, w in enumerate(data[i]):
        if j >= MAX_LEN:
            break
        X[i, j] = CHAR_TO_INDEX.get(encoded[i][j], CHAR_TO_INDEX['none'])
        XR[i, j] = CHAR_TO_INDEX.get(encoded_rand[i][j], CHAR_TO_INDEX['none'])
        Y[i, j] = CHAR_TO_INDEX.get(w, CHAR_TO_INDEX['none'])

In [96]:
X[0]

tensor([ 3, 17, 34,  7,  1, 33,  1, 33,  7, 32, 12,  3, 24, 14,  7, 16, 29, 32,
         8, 15, 24, 14,  7, 23, 19, 12, 21,  6, 31,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0])

### Задание №1

**Модель RNN**

In [9]:
class Network(torch.nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.embedding = torch.nn.Embedding(len(CHAR_TO_INDEX), 35)
        self.rnn = torch.nn.RNN(35, 128, batch_first=True)
        self.linear = torch.nn.Linear(128, len(INDEX_TO_CHAR))
        
    def forward(self, sentences, state=None):
        embed = self.embedding(sentences)
        o, s = self.rnn(embed)
        out = self.linear(o)
        return out

In [10]:
model = Network()
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=.05)

In [11]:
import time

In [100]:
def train_model(model, num_epochs, data, target):
    N = int(len(data) * 0.8)
    train, test = data[: N], data[N: ]
    y_train, y_test = target[: N], target[N: ]

    loss = torch.nn.CrossEntropyLoss()
    trainer = torch.optim.SGD(model.parameters(), lr=.05)

    for ep in range(num_epochs):
        
        train_iters, train_passed  = 0, 0
        train_loss, train_acc = 0., 0.
        start=time.time()
        
        model.train()
        for i in range(int(len(train) / 14)):
            X_batch = train[i * 14:(i + 1) * 14]
            Y_batch = y_train[i * 14:(i + 1) * 14].flatten()
            trainer.zero_grad()
            y_pred = model.forward(X_batch)
            y_pred = y_pred.view(-1, len(INDEX_TO_CHAR))
            l = loss(y_pred, Y_batch)
            l.backward()
            trainer.step()
            train_loss += l.item()
            train_acc += (y_pred.argmax(dim=1) == Y_batch).sum().item()
            train_iters += 1
            train_passed += Y_batch.shape[0]
        
        test_iters, test_passed  = 0, 0
        test_loss, test_acc = 0., 0.
        for i in range(int(len(test) / 14)):
            X_batch = test[i * 14:(i + 1) * 14]
            Y_batch = y_test[i * 14:(i + 1) * 14].flatten()
            y_pred = model(X_batch).view(-1, len(INDEX_TO_CHAR))
            l = loss(y_pred, Y_batch)
            test_loss += l.item()
            test_acc += (y_pred.argmax(dim=1) == Y_batch).sum().item()
            test_iters += 1
            test_passed += Y_batch.shape[0]
            
        train_acc_res = train_acc / train_passed
        test_acc_res = test_acc / test_passed
        print("ep: {}, time: {:.3f}, train_loss: {}, train_acc: {}, test_loss: {}, test_acc: {}".format(
            ep, time.time() - start, train_loss / train_iters, train_acc_res,
            test_loss / test_iters, test_acc_res)
        )

**Вариант №1. Смещение на 2**

In [101]:
model = Network()
train_model(model, 120, X, Y)

ep: 0, time: 0.080, train_loss: 2.9002789855003357, train_acc: 0.34732142857142856, test_loss: 2.1752326488494873, test_acc: 0.475
ep: 1, time: 0.086, train_loss: 2.0248415619134903, train_acc: 0.5294642857142857, test_loss: 1.9276302456855774, test_acc: 0.6151785714285715
ep: 2, time: 0.071, train_loss: 1.8318042308092117, train_acc: 0.6944196428571429, test_loss: 1.7724168300628662, test_acc: 0.7535714285714286
ep: 3, time: 0.078, train_loss: 1.6758822798728943, train_acc: 0.8033482142857142, test_loss: 1.6282222867012024, test_acc: 0.8133928571428571
ep: 4, time: 0.073, train_loss: 1.5291919857263565, train_acc: 0.8399553571428572, test_loss: 1.492543339729309, test_acc: 0.8330357142857143
ep: 5, time: 0.058, train_loss: 1.3935012519359589, train_acc: 0.8535714285714285, test_loss: 1.3683003783226013, test_acc: 0.8419642857142857
ep: 6, time: 0.066, train_loss: 1.2719790488481522, train_acc: 0.8569196428571428, test_loss: 1.2569966316223145, test_acc: 0.84375
ep: 7, time: 0.081, tra

In [ ]:
def decode(sent):
    x = torch.zeros((1, len(sent)), dtype=int)
    output = ''

    for j, w in enumerate(sent):
        x[0, j] = CHAR_TO_INDEX.get(w, CHAR_TO_INDEX['none'])
    
    o = model(x)[0]
    for w in o:
        ww = INDEX_TO_CHAR[torch.argmax(w)]        
        output += ww
    
    return output


Визуальная проверка результата

In [108]:
for i in range(5):
    print(f'encoded: {encoded[i]}\noriginal: {data[i]}\ndecoded: {decode(encoded[i])}\n=====')

encoded: орл ёбёб увоэч щжуфпэч ствдкн
original: мой дядя самых честных правил
decoded: мой дядя самых честных правил
=====
encoded: мреёв пж д ъхфмх йвпжоре
original: когда не в шутку занемог
decoded: когда не в шутку занемог
=====
encoded: рп хдвивфю ужгб йвуфвдкн
original: он уважать себя заставил
decoded: он уважать себя заставил
=====
encoded: к нхщъж дэёховфю пж оре
original: и лучше выдумать не мог
decoded: и лучше выдумать не мог
=====
encoded: жер сткожт ётхеко пвхмв
original: его пример другим наука
decoded: его пример другим наука
=====


**Вывод:** как визуальная проверка, так и метрики показывают, что наша модель в случае фиксированного сдвига показывает отличные результаты (0.99 test acc.)

**Вариант №2. Смещение от 2 до 5**

In [116]:
model = Network()
train_model(model, 500, XR, Y)

ep: 0, time: 0.093, train_loss: 2.8287146389484406, train_acc: 0.3658482142857143, test_loss: 2.1854909658432007, test_acc: 0.48839285714285713
ep: 1, time: 0.152, train_loss: 2.085079535841942, train_acc: 0.5042410714285714, test_loss: 2.0340269804000854, test_acc: 0.48928571428571427
ep: 2, time: 0.078, train_loss: 1.9674385040998459, train_acc: 0.5209821428571428, test_loss: 1.9513425827026367, test_acc: 0.5080357142857143
ep: 3, time: 0.075, train_loss: 1.8817001283168793, train_acc: 0.5332589285714285, test_loss: 1.8814446926116943, test_acc: 0.5223214285714286
ep: 4, time: 0.063, train_loss: 1.8093371540307999, train_acc: 0.5390625, test_loss: 1.8231950998306274, test_acc: 0.5223214285714286
ep: 5, time: 0.073, train_loss: 1.7501927465200424, train_acc: 0.5439732142857143, test_loss: 1.7751359343528748, test_acc: 0.5375
ep: 6, time: 0.083, train_loss: 1.7013835906982422, train_acc: 0.5604910714285715, test_loss: 1.7336634993553162, test_acc: 0.55
ep: 7, time: 0.069, train_loss: 1

In [117]:
for i in range(5):
    print(f'encoded: {encoded_rand[i]}\noriginal: {data[i]}\ndecoded: {decode(encoded[i])}\n=====')

encoded: ртн згзг хдрящ ыихцсящ уфдёмп
original: мой дядя самых честных правил
decoded: лои бядя оамыт черрныт ноааил
=====
encoded: пузие тй ж эшчпш метйсуз
original: когда не в шутку занемог
decoded: илгда не б чттис занемог
=====
encoded: ут шжелечб цйёд мецчежнр
original: он уважать себя заставил
decoded: нн свадать свбя еасравил
=====
encoded: л оцъыз еюжцпгхя рз псё
original: и лучше выдумать не мог
decoded: и лсчче быдулать не мог
=====
encoded: ижт уфмриф зфчжмр сдчод
original: его пример другим наука
decoded: вао ноимер вотвим натиа
=====


**Вывод:** в случае со случайным сдвигом букв (от 2 до 5) модель показывает более низкое качество. Это обусловлено тем, что сдвиг случаен и модель не может найти однозначную взаимосвязь между входными и выходными данными. Если бы каждая буква сдвигалась каждый раз на один и тот же шаг, то модель вполне могла значительно большей точности, так как была бы явная связь между "зашифрованными" и оригинальными символами. Вероятно, точность модели будет снижаться с ростом размера данных, т.к. связь между входными и выходными данными будет все менее явной.